In [1]:
import sqlite3
import json

In [2]:
conn = sqlite3.connect('../databases/CovidData.db')
cursor = conn.cursor()

In [3]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS covid_journals_data (
        id TEXT PRIMARY KEY,
        json_string TEXT
    )
''')

In [ ]:
with open('../data/combined_sources.json', 'r') as json_file:
    data = json.load(json_file)
    for item in data:
        cursor.execute("INSERT INTO covid_journals_data VALUES(?,?)", (item['id'],json.dumps(item),))

In [5]:
conn.commit()
conn.close()